This Notebook constains code that creates Table 1 of the paper, which reports the correlations between pairs of scores.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from sklearn.preprocessing import StandardScaler
import re

from knobs_functions import *

In [2]:
my_score_list = ['Reock', 'Polsby-Popper', 'cut edges', 'Dem seats', 'efficiency gap', 'mean-median', 'seat bias',
                  'competitive districts', 'average margin', 'county splits', 'MMD coalition']
print(my_score_list)

['Reock', 'Polsby-Popper', 'cut edges', 'Dem seats', 'efficiency gap', 'mean-median', 'seat bias', 'competitive districts', 'average margin', 'county splits', 'MMD coalition']


In [3]:
# Let's find the correlation table for all state-chamber combinations

import warnings
warnings.filterwarnings('ignore')

D = dict() # dictionary mapping (state, chamber) to the correlation table using all ensembles    

for state, chamber in state_chamber_list:
    print(state, chamber)
    D[(state, chamber)] = correlation_table(state, chamber, my_score_list=my_score_list)

FL congress
FL upper
FL lower
IL congress
IL upper
IL lower
MI congress
MI upper
MI lower
NC congress
NC upper
NC lower
NY congress
NY upper
NY lower
OH congress
OH upper
OH lower
WI congress
WI upper
WI lower


In [4]:
D[('WI', 'congress')]

,Reock,Polsby-Popper,cut edges,Dem seats,efficiency gap,mean-median,seat bias,competitive districts,average margin,county splits,MMD coalition
Reock,1.000000,0.382458,-0.278829,0.047829,-0.044182,-0.028568,-0.049319,0.002223,0.119166,-0.240068,NaN
Polsby-Popper,0.382458,1.000000,-0.867037,-0.225652,0.233802,0.240871,0.284845,-0.024370,0.364352,-0.929502,NaN
cut edges,-0.278829,-0.867037,1.000000,0.259139,-0.265516,-0.268315,-0.330383,0.014642,-0.367050,0.772040,NaN
Dem seats,0.047829,-0.225652,0.259139,1.000000,-0.997784,-0.625424,-0.841470,-0.060965,-0.369558,0.221627,NaN
efficiency gap,-0.044182,0.233802,-0.265516,-0.997784,1.000000,0.642657,0.843912,0.041882,0.402076,-0.229172,NaN
mean-median,-0.028568,0.240871,-0.268315,-0.625424,0.642657,1.000000,0.779955,-0.298611,0.639015,-0.235760,NaN
seat bias,-0.049319,0.284845,-0.330383,-0.841470,0.843912,0.779955,1.000000,0.056822,0.476848,-0.279533,NaN
competitive districts,0.002223,-0.024370,0.014642,-0.060965,0.041882,-0.298611,0.056822,1.000000,-0.478819,0.013018,NaN
average margin,0.119166,0.364352,-0.367050,-0.369558,0.402076,0.639015,0.476848,-0.478819,1.000000,-0.332281,NaN
county splits,-0.240068,-0.929502,0.772040,0.221627,-0.229172,-0.235760,-0.279533,0.013018,-0.332281,1.000000,NaN


In [5]:
# Let's average the correlation tables over the state-chamber combinations
# (we need some extra code so the average for each cell is over the non-Nan values)
sum_corr = pd.DataFrame(0.0, columns=my_score_list, index=my_score_list)
count_corr = pd.DataFrame(0, columns=my_score_list, index=my_score_list)

# Accumulate sum and count of non-NaN entries
for state_chamber in state_chamber_list:
    df = D[state_chamber]
    mask = df.notna()
    sum_corr += df.fillna(0)
    count_corr += mask.astype(int)

# Compute average of non-NaN values
avg_corr = sum_corr / count_corr
avg_corr = avg_corr.round(2)

In [6]:
avg_corr

,Reock,Polsby-Popper,cut edges,Dem seats,efficiency gap,mean-median,seat bias,competitive districts,average margin,county splits,MMD coalition
Reock,1.00,0.73,-0.72,0.01,-0.02,0.04,-0.01,-0.03,0.26,-0.63,0.06
Polsby-Popper,0.73,1.00,-0.91,0.01,-0.01,0.05,-0.01,-0.04,0.33,-0.95,0.06
cut edges,-0.72,-0.91,1.00,-0.01,0.01,-0.05,0.00,0.05,-0.34,0.81,-0.07
Dem seats,0.01,0.01,-0.01,1.00,-0.99,-0.24,-0.62,0.00,-0.09,-0.01,-0.05
efficiency gap,-0.02,-0.01,0.01,-0.99,1.00,0.24,0.60,-0.01,0.11,0.01,0.05
mean-median,0.04,0.05,-0.05,-0.24,0.24,1.00,0.37,-0.18,0.22,-0.05,0.07
seat bias,-0.01,-0.01,0.00,-0.62,0.60,0.37,1.00,-0.02,0.14,0.02,0.07
competitive districts,-0.03,-0.04,0.05,0.00,-0.01,-0.18,-0.02,1.00,-0.52,0.04,0.00
average margin,0.26,0.33,-0.34,-0.09,0.11,0.22,0.14,-0.52,1.00,-0.32,0.14
county splits,-0.63,-0.95,0.81,-0.01,0.01,-0.05,0.02,0.04,-0.32,1.00,-0.05


In [7]:
# Let's mark with * the score pairs for which the sign of the correlation is consistent across all state-chamber combinations.
avg_corr_marked = avg_corr.copy().round(2)
for score1 in my_score_list:
    for score2 in my_score_list:
        num_pos = len([1 for state_chamber in state_chamber_list if D[state_chamber].loc[score1, score2] > 0])
        num_neg = len([1 for state_chamber in state_chamber_list if D[state_chamber].loc[score1, score2] < 0])
        consistent_sign = 1 if num_neg == 0 else -1 if num_pos == 0 else 0
        if consistent_sign !=0:
            avg_corr_marked.loc[score1, score2] = f'*{avg_corr_marked.loc[score1, score2]}'
avg_corr_marked.to_excel('tables/avg_corr.xlsx')
avg_corr_marked

,Reock,Polsby-Popper,cut edges,Dem seats,efficiency gap,mean-median,seat bias,competitive districts,average margin,county splits,MMD coalition
Reock,*1.0,*0.73,*-0.72,0.01,-0.02,0.04,-0.01,-0.03,0.26,*-0.63,0.06
Polsby-Popper,*0.73,*1.0,*-0.91,0.01,-0.01,0.05,-0.01,-0.04,*0.33,*-0.95,0.06
cut edges,*-0.72,*-0.91,*1.0,-0.01,0.01,-0.05,0.0,0.05,*-0.34,*0.81,-0.07
Dem seats,0.01,0.01,-0.01,*1.0,*-0.99,-0.24,*-0.62,0.0,-0.09,-0.01,-0.05
efficiency gap,-0.02,-0.01,0.01,*-0.99,*1.0,0.24,*0.6,-0.01,0.11,0.01,0.05
mean-median,0.04,0.05,-0.05,-0.24,0.24,*1.0,0.37,-0.18,0.22,-0.05,0.07
seat bias,-0.01,-0.01,0.0,*-0.62,*0.6,0.37,*1.0,-0.02,0.14,0.02,0.07
competitive districts,-0.03,-0.04,0.05,0.0,-0.01,-0.18,-0.02,*1.0,*-0.52,0.04,0.0
average margin,0.26,*0.33,*-0.34,-0.09,0.11,0.22,0.14,*-0.52,*1.0,*-0.32,0.14
county splits,*-0.63,*-0.95,*0.81,-0.01,0.01,-0.05,0.02,0.04,*-0.32,*1.0,-0.05


In [8]:
# Process and output table for LaTex
def stack_string(a):
    l = re.split(r"[ -]+", a)
    if len(l) == 1:
        return l[0]
    else:
        return f'\\makecell{{{l[0]} \\\\ {l[1]}}}'
    
col_dict = {score: stack_string(score) for score in my_score_list}

avg_corr_marked.rename(columns=col_dict, inplace=True)
avg_corr_marked.to_latex('latex tables/correlations.tex', escape=False)

In [9]:
# Let's make a table showing the range of values over the state-chamber combinations
range_corr = pd.DataFrame(0, columns = my_score_list, index = my_score_list)
for score1 in my_score_list:
    for score2 in my_score_list:
        min_corr = min([D[state_chamber].loc[score1, score2] for state_chamber in state_chamber_list])
        max_corr = max([D[state_chamber].loc[score1, score2] for state_chamber in state_chamber_list])
        range_corr.loc[score1, score2] = f'({min_corr:.2f},{max_corr:.2f})'
range_corr.to_excel('tables/range_corr.xlsx')
range_corr

,Reock,Polsby-Popper,cut edges,Dem seats,efficiency gap,mean-median,seat bias,competitive districts,average margin,county splits,MMD coalition
Reock,"(1.00,1.00)","(0.38,0.87)","(-0.88,-0.28)","(-0.13,0.22)","(-0.23,0.13)","(-0.12,0.24)","(-0.16,0.21)","(-0.35,0.16)","(-0.03,0.59)","(-0.80,-0.24)","(-0.22,0.23)"
Polsby-Popper,"(0.38,0.87)","(1.00,1.00)","(-0.95,-0.87)","(-0.23,0.31)","(-0.33,0.23)","(-0.16,0.31)","(-0.21,0.28)","(-0.42,0.19)","(0.05,0.69)","(-0.98,-0.92)","(-0.31,0.26)"
cut edges,"(-0.88,-0.28)","(-0.95,-0.87)","(1.00,1.00)","(-0.32,0.26)","(-0.27,0.33)","(-0.27,0.14)","(-0.33,0.22)","(-0.17,0.38)","(-0.62,-0.07)","(0.77,0.88)","(-0.25,0.29)"
Dem seats,"(-0.13,0.22)","(-0.23,0.31)","(-0.32,0.26)","(1.00,1.00)","(-1.00,-0.97)","(-0.69,0.22)","(-0.84,-0.32)","(-0.13,0.29)","(-0.39,0.22)","(-0.31,0.22)","(-0.18,0.06)"
efficiency gap,"(-0.23,0.13)","(-0.33,0.23)","(-0.27,0.33)","(-1.00,-0.97)","(1.00,1.00)","(-0.23,0.69)","(0.31,0.84)","(-0.30,0.14)","(-0.22,0.40)","(-0.23,0.33)","(-0.05,0.17)"
mean-median,"(-0.12,0.24)","(-0.16,0.31)","(-0.27,0.14)","(-0.69,0.22)","(-0.23,0.69)","(1.00,1.00)","(-0.43,0.84)","(-0.69,0.53)","(-0.29,0.64)","(-0.31,0.16)","(-0.01,0.19)"
seat bias,"(-0.16,0.21)","(-0.21,0.28)","(-0.33,0.22)","(-0.84,-0.32)","(0.31,0.84)","(-0.43,0.84)","(1.00,1.00)","(-0.47,0.35)","(-0.30,0.58)","(-0.28,0.21)","(-0.31,0.26)"
competitive districts,"(-0.35,0.16)","(-0.42,0.19)","(-0.17,0.38)","(-0.13,0.29)","(-0.30,0.14)","(-0.69,0.53)","(-0.47,0.35)","(1.00,1.00)","(-0.67,-0.32)","(-0.19,0.42)","(-0.17,0.11)"
average margin,"(-0.03,0.59)","(0.05,0.69)","(-0.62,-0.07)","(-0.39,0.22)","(-0.22,0.40)","(-0.29,0.64)","(-0.30,0.58)","(-0.67,-0.32)","(1.00,1.00)","(-0.69,-0.05)","(-0.07,0.34)"
county splits,"(-0.80,-0.24)","(-0.98,-0.92)","(0.77,0.88)","(-0.31,0.22)","(-0.23,0.33)","(-0.31,0.16)","(-0.28,0.21)","(-0.19,0.42)","(-0.69,-0.05)","(1.00,1.00)","(-0.25,0.33)"
